##### 데이터 준비

In [1]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

훈련 : 테스트 : 검증 = 6 : 2 : 2

In [3]:
# validation set
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state=42)

In [4]:
print(train_input.shape)

(5197, 3)


In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


##### 결정 트리 

In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00920296, 0.00874734, 0.00955558, 0.01156354, 0.00852394]), 'score_time': array([0.00145745, 0.00107026, 0.00121546, 0.00142717, 0.00128222]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


fit-time : 모델을 훈련하는 시간 / score-time : 검증하는 시간

=> 5개 숫자 => 5-fold-cross-validation

In [8]:
# fit-time, score-time 출력안하고, test_score(검증 폴드 점수) 값만 반환 
from sklearn.model_selection import cross_val_score
scores1 = cross_val_score(dt, train_input, train_target)
print(scores1)

[0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]


In [9]:
import numpy as np
print(np.mean(scores['test_score']))
print(np.mean(scores1))

0.855300214703487
0.855300214703487


In [10]:
# 분류모델 분할기
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# 훈련세트 섞은 후, 10-폴드-교차검증
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


* 수동으로 매개변수 바꾸는 대신 그리드 서치나 랜덤 서치 사용하여 최적의 매개변수 찾기 !

#### 그리드 서치

In [12]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [13]:
dt = gs.best_estimator_  # 최적의 모델
print(dt.score(train_input, train_target))

0.9615162593804117


In [14]:
print(gs.best_params_)  # 최적의 매개변수

{'min_impurity_decrease': 0.0001}


In [15]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


첫번째 값(0.0001)이 가장 크다 !

In [16]:
# 인덱스 이용하여 최적의 매개변수 출력
best_index = np.argmax(gs.cv_results_['mean_test_score'])  # 가장 큰 값의 인덱스
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [17]:
# 여러개의 매개변수
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),   # 불순도 감소 최소량
          'max_depth' : range(5, 20, 1),   # 트리 깊이
          'min_samples_split' : range(2, 100, 10)}   # 노드 나누기 위한 최소 샘플 수

9 * 15 * 10 = 1350번 교차 검증 횟수

1350 * 5(fold) = 6750개 모델

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [19]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [20]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616


#### 랜덤 서치 

In [ ]:
from scipy.stats import uniform, randint
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter = 100, n_jobs = -1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f6190b42be0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f6190b8c2e0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f6190b42dc0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f6190b7f400>},
                   random_state=42)

In [ ]:
# 최적의 매개변수 조합
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
# 최고의 교차검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
# 테스트 성능
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
